# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
df.head()

loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0    0.1719       375.35           RENT     66000.0   
1    25000.0    0.2000       929.09       MORTGAGE    105000.0   
2    20000.0    0.2000       529.88       MORTGAGE     56000.0   
3    10000.0    0.1640       353.55           RENT     92000.0   
4    22000.0    0.1474       520.39       MORTGAGE     52000.0   

  verification_status   issue_d loan_status pymnt_plan    dti  delinq_2yrs  \
0     Source Verified  Mar-2019    low_risk          n  27.24          0.0   
1            Verified  Mar-2019    low_risk          n  20.23          0.0   
2            Verified  Mar-2019    low_risk          n  24.26          0.0   
3            Verified  Mar-2019    low_risk          n  31.44          0.0   
4        Not Verified  Mar-2019    low_risk          n  18.76          0.0   

   inq_last_6mths  open_acc  pub_rec  revol_bal  total_acc  \
0             0.0       8.0      0.0     1609.0       14.0   
1             0.0      17.0      1.0    18368.0       34.0   
2             0.0       8.0      0.0    13247.0       21.0   
3             1.0      10.0      1.0    17996.0       23.0   
4             1.0      14.0      0.0     9091.0       28.0   

  initial_list_status  out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  \
0                   w   10204.87       10204.87       355.29           355.29   
1                   w   24293.13       24293.13       873.53           873.53   
2                   w   19636.79       19636.79       485.44           485.44   
3                   w    9719.34        9719.34       335.33           335.33   
4                   w   21614.73       21614.73       484.36           484.36   

   total_rec_prncp  total_rec_int  total_rec_late_fee  recoveries  \
0           295.13          60.16                 0.0         0.0   
1           706.87         166.66                 0.0         0.0   
2           363.21         122.23                 0.0         0.0   
3           280.66          54.67                 0.0         0.0   
4           385.27          99.09                 0.0         0.0   

   collection_recovery_fee  last_pymnt_amnt next_pymnt_d  \
0                      0.0           375.35     May-2019   
1                      0.0           929.09     May-2019   
2                      0.0           529.88     May-2019   
3                      0.0           353.55     May-2019   
4                      0.0           520.39     May-2019   

   collections_12_mths_ex_med  policy_code application_type  acc_now_delinq  \
0                         0.0          1.0       Individual             0.0   
1                         0.0          1.0       Individual             0.0   
2                         0.0          1.0       Individual             0.0   
3                         0.0          1.0       Individual             0.0   
4                         0.0          1.0       Individual             0.0   

   tot_coll_amt  tot_cur_bal  open_acc_6m  open_act_il  open_il_12m  \
0         178.0      38199.0          3.0          4.0          2.0   
1         165.0     228119.0          3.0          3.0          2.0   
2           0.0      45684.0          2.0          3.0          2.0   
3           0.0      68784.0          0.0          4.0          1.0   
4           0.0     177745.0          0.0          1.0          0.0   

   open_il_24m  mths_since_rcnt_il  total_bal_il  il_util  open_rv_12m  \
0          4.0                 3.0       36590.0     59.0          1.0   
1          3.0                 4.0       42273.0     86.0          4.0   
2          3.0                 4.0       32437.0     75.0          2.0   
3          3.0                 9.0       50788.0     66.0          0.0   
4          2.0                14.0       16828.0     84.0          1.0   

   open_rv_24m  max_bal_bc  all_util  total_rev_hi_lim  inq_fi  total_cu_tl  \
0          2.0      1553.0      58.0            3700.0     1.0          0.0

In [5]:
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status            object
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_a

In [6]:
df_binary = pd.get_dummies(df, columns=["home_ownership", 
                                        "verification_status", 
                                        "issue_d", 
                                        "pymnt_plan", 
                                        "initial_list_status", 
                                        "next_pymnt_d", 
                                        "application_type", 
                                        "hardship_flag",
                                        "debt_settlement_flag"])
pd.reset_option('max_rows')
pd.reset_option('max_columns')
df_binary.dtypes

loan_amnt                      float64
int_rate                       float64
installment                    float64
annual_inc                     float64
loan_status                     object
                                ...   
next_pymnt_d_May-2019            uint8
application_type_Individual      uint8
application_type_Joint App       uint8
hardship_flag_N                  uint8
debt_settlement_flag_N           uint8
Length: 96, dtype: object

# Split the Data into Training and Testing

In [7]:
# Create our features
X = df_binary.drop("loan_status", axis=1)

# Create our target
y = df[target]

In [8]:
X

loan_amnt  int_rate  installment  annual_inc    dti  delinq_2yrs  \
0        10500.0    0.1719       375.35     66000.0  27.24          0.0   
1        25000.0    0.2000       929.09    105000.0  20.23          0.0   
2        20000.0    0.2000       529.88     56000.0  24.26          0.0   
3        10000.0    0.1640       353.55     92000.0  31.44          0.0   
4        22000.0    0.1474       520.39     52000.0  18.76          0.0   
...          ...       ...          ...         ...    ...          ...   
68812    10000.0    0.1502       346.76     26000.0   9.60          0.0   
68813    12000.0    0.2727       368.37     63000.0  29.07          0.0   
68814     5000.0    0.1992       185.62     52000.0  14.86          0.0   
68815    40000.0    0.0646      1225.24    520000.0   9.96          0.0   
68816    16000.0    0.1131       350.36     72000.0   7.02          2.0   

       inq_last_6mths  open_acc  pub_rec  revol_bal  ...  issue_d_Mar-2019  \
0                 0.0       8.0      0.0     1609.0  ...                 1   
1                 0.0      17.0      1.0    18368.0  ...                 1   
2                 0.0       8.0      0.0    13247.0  ...                 1   
3                 1.0      10.0      1.0    17996.0  ...                 1   
4                 1.0      14.0      0.0     9091.0  ...                 1   
...               ...       ...      ...        ...  ...               ...   
68812             0.0       9.0      0.0     2684.0  ...                 0   
68813             0.0       8.0      0.0    13314.0  ...                 0   
68814             0.0       5.0      1.0     3715.0  ...                 0   
68815             1.0      21.0      0.0    59529.0  ...                 0   
68816             0.0      12.0      1.0    11882.0  ...                 0   

       pymnt_plan_n  initial_list_status_f  initial_list_status_w  \
0                 1                      0                      1   
1                 1                      0                      1   
2                 1                      0                      1   
3                 1                      0                      1   
4                 1                      0                      1   
...             ...                    ...                    ...   
68812             1                      0                      1   
68813             1                      0                      1   
68814             1                      0                      1   
68815             1                      1                      0   
68816             1                      0                      1   

       next_pymnt_d_Apr-2019  next_pymnt_d_May-2019  \
0                          0                      1   
1                          0                      1   
2                          0                      1   
3                          0                      1   
4                          0                      1   
...                      ...                    ...   
68812                      0                      1   
68813                      0                      1   
68814                      0                      1   
68815                      0                      1   
68816                      0                      1   

       application_type_Individual  application_type_Joint App  \
0                                1                           0   
1                                1                           0   
2                                1                           0   
3                                1                           0   
4                                1                           0   
...                            ...                         ...   
68812                            1                           0   
68813                            1                           0   
68814                            1                           0   
68815              

In [9]:
X.describe()

loan_amnt      int_rate   installment    annual_inc           dti  \
count  68817.000000  68817.000000  68817.000000  6.881700e+04  68817.000000   
mean   16677.594562      0.127718    480.652863  8.821371e+04     21.778153   
std    10277.348590      0.048130    288.062432  1.155800e+05     20.199244   
min     1000.000000      0.060000     30.890000  4.000000e+01      0.000000   
25%     9000.000000      0.088100    265.730000  5.000000e+04     13.890000   
50%    15000.000000      0.118000    404.560000  7.300000e+04     19.760000   
75%    24000.000000      0.155700    648.100000  1.040000e+05     26.660000   
max    40000.000000      0.308400   1676.230000  8.797500e+06    999.000000   

        delinq_2yrs  inq_last_6mths      open_acc       pub_rec  \
count  68817.000000    68817.000000  68817.000000  68817.000000   
mean       0.217766        0.497697     12.587340      0.126030   
std        0.718367        0.758122      6.022869      0.336797   
min        0.000000        0.000000      2.000000      0.000000   
25%        0.000000        0.000000      8.000000      0.000000   
50%        0.000000        0.000000     11.000000      0.000000   
75%        0.000000        1.000000     16.000000      0.000000   
max       18.000000        5.000000     72.000000      4.000000   

           revol_bal  ...  issue_d_Mar-2019  pymnt_plan_n  \
count   68817.000000  ...      68817.000000       68817.0   
mean    17604.142828  ...          0.177238           1.0   
std     21835.880400  ...          0.381873           0.0   
min         0.000000  ...          0.000000           1.0   
25%      6293.000000  ...          0.000000           1.0   
50%     12068.000000  ...          0.000000           1.0   
75%     21735.000000  ...          0.000000           1.0   
max    587191.000000  ...          1.000000           1.0   

       initial_list_status_f  initial_list_status_w  next_pymnt_d_Apr-2019  \
count           68817.000000           68817.000000           68817.000000   
mean                0.123879               0.876121               0.383161   
std                 0.329446               0.329446               0.486161   
min                 0.000000               0.000000               0.000000   
25%                 0.000000               1.000000               0.000000   
50%                 0.000000               1.000000               0.000000   
75%                 0.000000               1.000000               1.000000   
max                 1.000000               1.000000               1.000000   

       next_pymnt_d_May-2019  application_type_Individual  \
count           68817.000000                 68817.000000   
mean                0.616839                     0.860340   
std                 0.486161                     0.346637   
min                 0.000000                     0.000000   
25%                 0.000000                     1.000000   
50%                 1.000000                     1.000000   
75%                 1.000000                     1.000000   
max                 1.000000                     1.000000   

       application_type_Joint App  hardship_flag_N  debt_settlement_flag_N  
count                68817.000000          68817.0                 68817.0  
mean                     0.139660              1.0                     1.0  
std                      0.346637              0.0                     0.0  
min                      0.000000              1.0                     1.0  
25%                      0.000000              1.0                     1.0  
50%                      0.000000              1.0                     1.0  
75%                      0.000000              1.0                     1.0  
max                      1.000000              1.0                     1.0  

[8 rows x 95 columns]

In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1)
Counter(y_train)

Counter({'loan_status': 1})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [13]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

bac = balanced_accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {bac}")

Accuracy Score : 0.640324421824783


In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

Predicted high_risk  Predicted low_risk
Actual high_risk                   67                  34
Actual low_risk                  6546               10558

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Naive Random Oversampling Classification Report")
print("_______________________________________________")
print(classification_report_imbalanced(y_test, y_pred))

Naive Random Oversampling Classification Report
_______________________________________________
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.62      0.02      0.64      0.41       101
   low_risk       1.00      0.62      0.66      0.76      0.64      0.41     17104

avg / total       0.99      0.62      0.66      0.76      0.64      0.41     17205



### SMOTE Oversampling

In [17]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'loan_status': 1})

In [18]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

In [19]:
# Calculated the balanced accuracy score

bac = balanced_accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {bac}")

Accuracy Score : 0.6514992150524688


In [20]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

Predicted high_risk  Predicted low_risk
Actual high_risk                   62                  39
Actual low_risk                  5317               11787

In [21]:
# Print the imbalanced classification report
print("SMOTE Oversampling Classification Report")
print("________________________________________")
print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling Classification Report
________________________________________
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.69      0.02      0.65      0.42       101
   low_risk       1.00      0.69      0.61      0.81      0.65      0.43     17104

avg / total       0.99      0.69      0.61      0.81      0.65      0.43     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [22]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [23]:
# Train the Logistic Regression model using the resampled data

model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

In [24]:
# Calculated the balanced accuracy score
bac = balanced_accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {bac}")

Accuracy Score : 0.5443538770387797


In [25]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

Predicted high_risk  Predicted low_risk
Actual high_risk                   70                  31
Actual low_risk                 10337                6767

In [26]:
# Print the imbalanced classification report
print("Undersampling Classification Report")
print("___________________________________")
print(classification_report_imbalanced(y_test, y_pred))

Undersampling Classification Report
___________________________________
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.40      0.01      0.52      0.28       101
   low_risk       1.00      0.40      0.69      0.57      0.52      0.27     17104

avg / total       0.99      0.40      0.69      0.56      0.52      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [27]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'loan_status': 1})

In [28]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)

In [29]:
# Calculated the balanced accuracy score
bac = balanced_accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {bac}")

Accuracy Score : 0.6449163069955265


In [30]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

Predicted high_risk  Predicted low_risk
Actual high_risk                   73                  28
Actual low_risk                  7405                9699

In [32]:
# Print the imbalanced classification report
print("Combination Sampling Classification Report")
print("__________________________________________")
print(classification_report_imbalanced(y_test, y_pred))

Combination Sampling Classification Report
__________________________________________
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.57      0.02      0.64      0.42       101
   low_risk       1.00      0.57      0.72      0.72      0.64      0.40     17104

avg / total       0.99      0.57      0.72      0.72      0.64      0.40     17205

